This tutorial can be downloaded [link](https://github.com/west-code-development/West/raw/master/Doc/tutorials/basic/basic_005.ipynb).

# Intro Tutorial 5: Quantum Defect Embedding Theory (QDET) Calculations

This tutorial shows how to compute excitation energies of the negatively charged nitrogen-vacancy defect (NV$^-$) in diamond using the Quantum Defect Embedding Theory (QDET). In QDET, a set of selected Kohn-Sham (KS) orbitals forms the active space. The effective Hamiltonian that describes excitations within the active space is formulated as:

$$
H^{\text{eff}} = \sum_{ij}^A t^{\text{eff}}_{ij}\hat{a}_i^\dagger \hat{a}_j + \frac{1}{2} \sum_{ijkl}^A v^{\text{eff}}_{ijkl} \hat{a}_i^\dagger \hat{a}_j^\dagger \hat{a}_l \hat{a}_k,
$$

where $t^{\text{eff}}$ and $v^{\text{eff}}$ are the effective one- and two-body terms, respectively. Exact diagonalization of the effective Hamiltonian yields the excitations of the spin defect. More details about the method, and definitions for $t^{\text{eff}}$ and $v^{\text{eff}}$ can be found in [N. Sheng, C. Vorwerk, M. Govoni, and G. Galli, J. Chem. Theory Comput. 18, 3512 (2022)](https://doi.org/10.1021/acs.jctc.2c00240).

QDET calculations are performed in two separate steps:
1. The parameters $t^{\text{eff}}$ and $v^{\text{eff}}$ are calculated from first principles and written to file.
2. The effective Hamiltonian is constructed from the parameters read from file. Exact diagonalization is used to find the lowest eigenvalues and eigenvectors of the effective Hamiltonian.

## Step 1: Parameters of the effective Hamiltonian

The parameters of the effective Hamiltonian $H^{\text{eff}}$ are calculated from first principles by:
- 1.1 Carrying out a mean-field starting point calculation using density-functional theory (DFT)
- 1.2 Computing the projective dielectric eigendcomposition (PDEP)
- 1.3 Computing $t^{\text{eff}}$ and $v^{\text{eff}}$ for a chosen active space, and saving them to file

### Step 1.1:  Mean-field starting point

As a first step, we perform the mean-field electronic structure calculation within DFT using the [Quantum ESPRESSO](https://www.quantum-espresso.org/) code.

Download the following files in your working directory:

In [1]:
%%bash
wget -N -q https://west-code.org/doc/training/nv_diamond_63/pw.in
wget -N -q http://www.quantum-simulation.org/potentials/sg15_oncv/upf/C_ONCV_PBE-1.2.upf
wget -N -q http://www.quantum-simulation.org/potentials/sg15_oncv/upf/N_ONCV_PBE-1.2.upf

We inspect the `pw.in` file, the input for the `pw.x` code:

In [2]:
%%bash
cat pw.in

&CONTROL
calculation = 'scf'
pseudo_dir = './'
/
&SYSTEM
input_dft = 'PBE'
ibrav = 0
ecutwfc = 50
tot_charge = -1
nspin = 1
nbnd = 176
occupations = 'from_input'
nat = 63
ntyp = 2
/
&ELECTRONS
diago_full_acc = .true.
/
K_POINTS gamma
CELL_PARAMETERS angstrom
7.136012  0.000000  0.000000
0.000000  7.136012  0.000000
0.000000  0.000000  7.136012
ATOMIC_SPECIES
C  12.01099968  C_ONCV_PBE-1.2.upf
N  14.00699997  N_ONCV_PBE-1.2.upf
ATOMIC_POSITIONS crystal
C    0.99996000  0.99996000  0.99996000
C    0.12495000  0.12495000  0.12495000
C    0.99905000  0.25039000  0.25039000
C    0.12350000  0.37499000  0.37499000
C    0.25039000  0.99905000  0.25039000
C    0.37499000  0.12350000  0.37499000
C    0.25039000  0.25039000  0.99905000
C    0.37499000  0.37499000  0.12350000
C    0.00146000  0.00146000  0.50100000
C    0.12510000  0.12510000  0.62503000
C    0.00102000  0.24944000  0.74960000
C    0.12614000  0.37542000  0.87402000
C    0.24944000  0.00102000  0.74960000
C    0.37542000  0.12614

We run `pw.x` on 2 cores.

In [ ]:
%%bash
mpirun -n 2 pw.x -i pw.in > pw.out

We note that the calculation is spin unpolarized (i.e., `nspin=1`) in order to reduce spin contamination. The parameter `ecutwfc` and the size of the supercell may be increased to converge the results.

### Step 1.2: Calculation of dielectric screening

As for GW calculations with WEST, we first determine the dielectric screening before we can proceed to calculate the excitations of the spin defect. In WEST, the dielectric screening is obtained from the projective dielectric eigendecomposition (PDEP). The calculation with `wstat.x` requires an input file `wstat.in`.

Download the following file in your working directory:

In [3]:
%%bash
wget -N -q https://west-code.org/doc/training/nv_diamond_63/wstat.in

Once again, we can have a look at the input file:

In [4]:
%%bash
cat wstat.in

wstat_control:
  wstat_calculation: S
  n_pdep_eigen: 512
  trev_pdep: 0.00001


As we can see, there are no input parameters in `wstat.in` specific to QDET. The number of PDEPs, `n_pdep_eigen`, may be increased to converge the results.

We run `wstat.x` on 2 cores.

In [ ]:
%%bash
mpirun -n 2 wstat.x -i wstat.in > wstat.out

### Step 1.3: QDET calculation

The calculation of the paramters for the effective Hamiltonian is carried out by using the `wfreq.x` program. The program will: (i) compute the $G_0W_0$ quasiparticle energies, (ii) compute the partially screened Coulomb potential, and (iii) finally compute the parameters of the effective Hamiltonian.

Download the following file in your working directory:

In [5]:
%%bash
wget -N -q https://west-code.org/doc/training/nv_diamond_63/wfreq.in

Let us again inspect the file `wfreq.in`:

In [6]:
%%bash
cat wfreq.in

wstat_control:
  wstat_calculation: S
  n_pdep_eigen: 512
  trev_pdep: 0.00001

wfreq_control:
  wfreq_calculation: XWGQH
  macropol_calculation: C
  l_enable_off_diagonal: true
  n_pdep_eigen_to_use: 512
  qp_bands: [87, 122, 123, 126, 127, 128]
  n_refreq: 300
  ecut_refreq: 2.0


The QDET calculation is triggered by the keyword `H` (for effective Hamiltonian) in the variable `wfreq_calculation`. The KS states that define the active space are selected with the variable `qp_bands` (in our case states 87, 122, 123, 126, 127, and 128). The active space may be chosen to be a set of localized defect orbitals in the supercell. See [Tutorial 6](https://github.com/west-code-development/West/raw/master/Doc/tutorials/basic/basic_006.ipynb) for the identification of the localized orbitals.

For QDET, it is mandatory to set the flag `l_enable_off_diagonal : true` which activates the calculation of off-diagonal entries of the GW self-energy matrix.

We run `wfreq.x` on 2 cores.

In [ ]:
%%bash
mpirun -n 2 wfreq.x -i wfreq.in > wfreq.out

If the reader does NOT have the computational resources to run the calculation, the WEST output file needed for the next step can be directly downloaded as:

In [7]:
%%bash
mkdir -p west.wfreq.save
wget -N -q https://west-code.org/doc/training/nv_diamond_63/wfreq.json -O west.wfreq.save/wfreq.json

## Step 2: Exact diagonalization of the effective Hamiltonian

The effective Hamiltonian is constructed and diagonalized using the [WESTpy Python package](https://west-code.org/doc/westpy/latest/).

In [8]:
from westpy.qdet import QDETResult

# construct object for effective Hamiltonian
effective_hamiltonian = QDETResult(filename='west.wfreq.save/wfreq.json')

# diagonalize Hamiltonian
solution = effective_hamiltonian.solve()

 
 _    _ _____ _____ _____            
| |  | |  ___/  ___|_   _|           
| |  | | |__ \ `--.  | |_ __  _   _  
| |/\| |  __| `--. \ | | '_ \| | | | 
\  /\  / |___/\__/ / | | |_) | |_| | 
 \/  \/\____/\____/  \_/ .__/ \__, | 
                       | |     __/ | 
                       |_|    |___/  
 
WEST version     :  5.5.0
Today            :  2024-02-02 13:54:03.951228
-----------------------------------------------------
Building effective Hamiltonian...
nspin: 1
occupations: [[2. 2. 2. 2. 1. 1.]]


diag[1RDM - 1RDM(GS)]                                 
  E [eV] char                    87    122    123    126   127   128
0  0.000   3-                 0.000  0.000  0.000  0.000 0.000 0.000
1  0.436   1-                -0.001 -0.009 -0.018 -0.067 0.004 0.091
2  0.436   1-                -0.001 -0.009 -0.018 -0.067 0.092 0.002
3  1.251   1-                -0.002 -0.019 -0.023 -0.067 0.054 0.057
4  1.939   3-                -0.003 -0.010 -0.127 -0.860 1.000 0.000
5  1.940   3-                -0.003 -0.010 -0.127 -0.860 0.000 1.000
6  2.935   1-                -0.000 -0.032 -0.043 -0.855 0.929 0.002
7  2.936   1-                -0.000 -0.032 -0.043 -0.855 0.002 0.929
8  4.661   1-                -0.006 -0.054 -0.188 -1.672 0.960 0.960
9  5.080   3-                -0.014 -0.698 -0.213 -0.075 1.000 0.000

-----------------------------------------------------


Calling the function `solve()` of the object `QDETResult` writes the excitation energies (in eV), spin multiplicity and relative occupation (compared to the occupation of the ground state) to screen.

Let us consider the first excited state (`# 1`) as an example: It has an energy of `0.436 eV` above the ground state, has spin multiplicity `1` (i.e., it is a singlet), and is two-fold degenerate because state `# 2` has the same energy. From the one-body reduced density matrix (1RDM) analysis we see that 1 electron has been subtracted majorily from state 126 and it has been promoted to state 127 and state 128.

More detailed information about the excitations is stored in the dictionary `solution`:

In [9]:
for key in solution.keys() : 
    print(f'{key}:{solution[key]}')

nstates:10
nelec:(5, 5)
norb:6
evs_au:[4.18773738 4.20375853 4.20377099 4.23369325 4.25900775 4.25903557
 4.29560564 4.29564459 4.35901051 4.37440862]
evs:[0.         0.43595765 0.4362967  1.25052286 1.93936548 1.94012256
 2.93524492 2.93630467 4.66057925 5.07958334]
evcs:[FCIvector([[-3.29674537e-15, -7.07106780e-01,  2.04533335e-05,
             1.18588538e-06, -3.83162589e-06, -3.47414437e-06],
           [ 7.07106780e-01, -9.51376896e-15, -1.74794368e-05,
            -1.22602660e-05,  3.91099022e-06, -9.13376476e-06],
           [-2.04533335e-05,  1.74794368e-05,  2.67694696e-15,
            -4.58827377e-08, -4.86043103e-09,  2.27797193e-09],
           [-1.18588539e-06,  1.22602660e-05,  4.58827411e-08,
             2.25803817e-15, -1.73431686e-09, -1.34489761e-08],
           [ 3.83162590e-06, -3.91099022e-06,  4.86042931e-09,
             1.73431285e-09,  5.13921145e-16,  1.02893774e-08],
           [ 3.47414437e-06,  9.13376476e-06, -2.27797523e-09,
             1.34489754e-08,

Explanation of the keys:

- `nstates` : the number of excitations that have been calculated.
- `evs` : excitation energies relative to the ground state (in eV).
- `evs_au`: absolute energies of the many-body states (in Hartree).
- `mults`: spin multiplicity of each state.
- `rdm1s`: spin-summed reduced 1-body density matrix for each many-body state.
- `evcs`: many-body state in 2nd quantization (eigenvector of the effective Hamiltonian).